# 言語理解（機械学習による方法：スロット値推定）
ここでは、言語理解を機械学習による方法で実装します。サンプルのデータセットとしてレストラン検索と天気案内のユーザ発話およびそのアノテーションデータを用意しました。このデータでは既にMeCab（標準搭載の辞書）で単語分割が行われています。それぞれのドメインにおいてスロット値推定を実装します。

## 事前の設定
- 必要なpythonライブラリ
    - scikit-learn
    - numpy
    - gensim
    - sklearn-crfsuite
- MeCabのインストール
- MeCabをpythonで使用するためのmecab-pythonのインストール
- sklearn-crfsuiteとは
    - scikit-learnのようなスタイルでCRFを学習するためのライブラリ（crfsuiteのラッパー）
    - https://sklearn-crfsuite.readthedocs.io/en/latest/index.html

## 機械学習の手順

- 処理の手順は以下の通りです。
    - データを読み込む
    - 学習とテストデータに分割（80対20）
    - 入力データを特徴量に変換する
    - 出力データをラベルに変換する
    - 機械学習ライブラリを用いてモデルを学習する
    - 学習したモデルをテストデータで評価する

In [1]:
# 必要なラブラリを読み込む

import sys, os
import re
import numpy as np
import pickle

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import classification_report

import MeCab

In [2]:
# 変数の設定

NUM_TRAIN = 80      # 100個のデータのうち最初の80個を学習に利用
NUM_TEST = 20       # 100個のデータのうち残りの20個をテストに利用

In [3]:
# データを読み込む

# 使用するドメインを設定する
USED_DATASET = 1    # 0: 'restaurant', 1: 'weather'

if USED_DATASET == 0:
    # レストランデータ
    with open('./data/slu-restaurant-annotated.csv', 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    # 正解ラベルの一覧
    LABELS = [
        'B-budget', 'I-budget',
        'B-mood', 'I-mood',
        'B-loc', 'I-loc',
        'B-genre', 'I-genre',
        'B-style', 'I-style',
        'B-rate', 'I-rate',
        'O'
    ]

    SAVED_MODEL = './data/slu-slot-restaurant-crf.model'

elif USED_DATASET == 1:
    # 天気案内
    with open('./data/slu-weather-annotated.csv', 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    LABELS = [
        'B-wh', 'I-wh',
        'B-place', 'I-place',
        'B-when', 'I-when',
        'B-type', 'I-type',
        'B-temp', 'I-temp',
        'B-rain', 'I-rain',
        'B-wind', 'I-wind',
        'O'
    ]

    SAVED_MODEL = './data/slu-slot-weather-crf.model'

# 学習とテストデータに分割（80対20）
# 注）本来は交差検定を行うことが望ましい
lines_train = lines[:NUM_TRAIN]
lines_test = lines[NUM_TRAIN:]

data_train = []
for line in lines_train:

    # 既に分割済みの単語系列を使用
    d = line.strip().split(',')[2].split('/')

    # 正解ラベルの系列
    a = line.strip().split(',')[3].split('/')
    
    # 入力単語系列と正解ラベル系列のペアを格納
    data_train.append([d, a])

# テストデータも同様
data_test = []
for line in lines_test:
    d = line.strip().split(',')[2].split('/')
    a = line.strip().split(',')[3].split('/')
    data_test.append([d, a])

# 最初のデータだけ表示
print(data_train[0])
print(data_test[0])


[['左京', '区', 'の', '明日', 'の', '天気', 'を', '教え', 'て'], ['B-place', 'I-place', 'O', 'B-when', 'O', 'O', 'O', 'O', 'O']]
[['兵庫', 'の', '雨', 'は', '止ん', 'で', 'い', 'ます', 'か'], ['B-place', 'O', 'B-type', 'I-type', 'I-type', 'O', 'O', 'O', 'O']]


In [4]:
# 入力と正解ラベルで別々のデータにする
# 注）sklearn-crfsuiteのOne-hot表現は単語データをそのまま入力すればよい
train_x = [d[0] for d in data_train]
train_y = [d[1] for d in data_train]

# 学習サンプル数
print(len(train_x))

80


In [5]:
# CRFによる学習
# 注）実際にはパラメータの調整が必要だが今回は行わない
clf = sklearn_crfsuite.CRF()
clf.fit(train_x, train_y) 

# 学習済みモデルを保存
pickle.dump(clf, open(SAVED_MODEL, 'wb'))

In [6]:
# テストデータの作成
test_x = [d[0] for d in data_test]
test_y = [d[1] for d in data_test]

# テストサンプル数
print(len(test_x))

20


In [7]:
# テストデータでの評価
predict_y = clf.predict(test_x)

# 評価結果を表示
target_names = ['restaurant', 'weather']
print(metrics.flat_classification_report(test_y, predict_y, labels=LABELS))


              precision    recall  f1-score   support

        B-wh       0.83      0.62      0.71         8
        I-wh       1.00      0.69      0.82        13
     B-place       1.00      0.88      0.93         8
     I-place       0.00      0.00      0.00         0
      B-when       0.81      0.93      0.87        14
      I-when       0.00      0.00      0.00         1
      B-type       1.00      0.90      0.95        10
      I-type       0.00      0.00      0.00         2
      B-temp       0.00      0.00      0.00         5
      I-temp       0.00      0.00      0.00         3
      B-rain       1.00      1.00      1.00         1
      I-rain       1.00      1.00      1.00         1
      B-wind       0.00      0.00      0.00         3
      I-wind       0.00      0.00      0.00         3
           O       0.77      0.99      0.87        82

   micro avg       0.82      0.82      0.82       154
   macro avg       0.49      0.47      0.48       154
weighted avg       0.74   

C:\Users\inoue\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['B-wh', 'I-wh', 'B-place', 'I-place', 'B-when', 'I-when', 'B-type', 'I-type', 'B-temp', 'I-temp', 'B-rain', 'I-rain', 'B-wind', 'I-wind', 'O'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\Users\inoue\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\inoue\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _war

In [8]:
# 自由なデータで試してみる

# 入力データ
#input_data = 'この辺りであっさりした四川料理のお店に行きたい'
input_data = '横浜の今日の天気'

# MeCabによる分割と特徴量抽出
m = MeCab.Tagger ("-Owakati")
words_input = m.parse(input_data).strip().split(' ')

# 予測
predict_y = clf.predict([words_input])[0]
for word, tag in zip(words_input, predict_y):
    print(word + "\t" + tag)

横浜	B-place
の	O
今日	B-when
の	O
天気	O
